In [12]:
import pandas as pd
import numpy as np

In [13]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

In [14]:
np.random.seed(42)

In [15]:
def gerar_dados_clientes(n=1000):
    # --- Geração de Features Aleatórias ---
    
    # Idade: Mistura de jovens (18-50) e idosos (51-90)
    idade = np.concatenate([np.random.randint(18, 50, int(n*0.6)), np.random.randint(51, 90, int(n*0.4))])
    
    # Renda Anual (distribuição variada)
    renda = np.random.normal(35000, 15000, n)
    renda = np.maximum(renda, 1320 * 12) # Garantir pelo menos um salário mínimo anual
    
    # Beneficiário INSS (Mais provável se idade > 60)
    beneficiario = [1 if (i > 60 and np.random.rand() > 0.2) else 0 for i in idade]
    
    # Tempo médio no app (minutos por sessão). Idosos tendem a demorar mais se tiverem dificuldade
    tempo_app = []
    for i in idade:
        if i > 60:
            tempo_app.append(np.random.normal(12, 4)) # Média 12 min
        else:
            tempo_app.append(np.random.normal(3, 1.5)) # Média 3 min
    tempo_app = np.maximum(tempo_app, 1) # Mínimo 1 minuto
    
    # Percentual de erro de digitação/input
    # Jovens erram pouco, idosos com dificuldade erram mais
    erro_pct = []
    for i in idade:
        base_erro = np.random.uniform(0, 5)
        if i > 60:
            base_erro += np.random.uniform(5, 25) # Adiciona dificuldade
        erro_pct.append(base_erro)

    # Região (0: Capital, 1: Interior/Pequena Cidade)
    regiao = np.random.choice(['Capital', 'Interior'], n)
    
    # Pagamento em dia (Booleano: 1 = Sim, 0 = Não)
    # A Dona Nair paga em dia, então vamos garantir que muitos idosos paguem em dia
    pagamento_em_dia = []
    for i in idade:
        chance = 0.9 if i > 50 else 0.75 # Idosos costumam ser mais fiéis ao pagamento
        pagamento_em_dia.append(1 if np.random.rand() < chance else 0)

    # Histórico de contato com suporte (1 = Sim, 0 = Não)
    contato_suporte = [1 if (e > 15 or t > 10) else 0 for e, t in zip(erro_pct, tempo_app)]

    # --- Criação do DataFrame ---
    df = pd.DataFrame({
        'idade': idade,
        'renda_anual': renda.round(2),
        'beneficiario_inss': beneficiario,
        'tem_emprestimo': np.random.choice([0, 1], n),
        'tem_seguro': np.random.choice([0, 1], n),
        'historico_problemas': np.random.choice([0, 1], n, p=[0.8, 0.2]),
        'contato_suporte': contato_suporte,
        'visita_agente': np.random.choice([0, 1], n, p=[0.9, 0.1]),
        'tempo_medio_app_min': np.round(tempo_app, 1),
        'regiao': regiao,
        'erro_digitacao_pct': np.round(erro_pct, 1),
        'pagamento_em_dia': pagamento_em_dia
    })

    # --- Definindo a Target (Quem precisa de ajuda?) ---
    # Lógica de negócio: Idade avançada E (Erro alto OU Tempo alto de app)
    conditions = [
        (df['idade'] >= 60) & ((df['erro_digitacao_pct'] > 10) | (df['tempo_medio_app_min'] > 8)),
        (df['contato_suporte'] == 1) & (df['erro_digitacao_pct'] > 15)
    ]
    choices = [1, 1] # 1 = Tem Dificuldade Digital / Precisa de Atendimento Humano
    df['dificuldade_digital'] = np.select(conditions, choices, default=0)
    
    return df

In [16]:
# Gerar o dataset
df_clientes = gerar_dados_clientes(640000) #10% 6.4 Mi

# Visualizar a "Dona Nair" (um exemplo positivo)
print("Exemplo de Perfil Vulnerável (Dona Nair):")
print(df_clientes[df_clientes['dificuldade_digital'] == 1].head(1))

Exemplo de Perfil Vulnerável (Dona Nair):
        idade  renda_anual  beneficiario_inss  tem_emprestimo  tem_seguro  historico_problemas  contato_suporte  visita_agente  tempo_medio_app_min   regiao  erro_digitacao_pct  pagamento_em_dia  dificuldade_digital
384000     65      15840.0                  0               0           0                    0                1              0                 11.5  Capital                19.1                 1                    1


In [ ]:
df_clientes.to_csv("./data/data.csv", index=False)